In [113]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import numpy as np

In [114]:
df = getting_spreadsheet('z:\\Python Projects\\week_5.csv') # Retrieving Spreadsheet
df.drop(['Unnamed: 0', 'result'],inplace=True, axis=1)
df.replace(np.nan, '', inplace=True)
df, year = loading_538_predictions(df)
df

,date,team,win%,odds,implied%,pick,w/l
0,10/3/2019,L.A. Rams,45%,105,0.487805,,
1,,Seattle,55%,-125,0.555556,,
2,10/6/2019,Arizona,39%,145,0.408163,,
3,,Cincinnati,61%,-165,0.622642,,
4,10/6/2019,Atlanta,37%,180,0.357143,Atlanta,
5,,Houston,63%,-220,0.6875,,
6,10/6/2019,Baltimore,58%,-180,0.642857,,
7,,Pittsburgh,42%,160,0.384615,Pittsburgh,
8,10/6/2019,Buffalo,32%,140,0.416667,,
9,,Tennessee,68%,-160,0.615385,Tennessee,


In [115]:
def loading_538_predictions(df):

    #df_columns = ['team', 'win%', 'date', 'result']
    #df = pd.DataFrame(columns=df_columns)
    
    output_file = 'z:\python_projects\aaa.exe'
    year = 2019
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2019-nfl-predictions/games/?ex_cid=rrpromo'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    # Finding year of predictions
    for timestamp in soup.findAll('div', attrs={'class': 'container'}):
        for year in timestamp.findAll('div', attrs={'id': 'intro'}):
            year = int(year.h1.get_text()[0:5])

    # Gathering data
    for week in soup.findAll('section', attrs={'class': 'week'}):
        if week.get_text()[5] == '5':
            for date in week.findAll('div', attrs={'class': 'days'}):
                for day in date.findAll('div', attrs={'class': 'day'}):  
                    for game in day.findAll('div', attrs={'class': 'game'}):

                        # Finding date of each game  
                        for h4 in day.findAll('h4', attrs={'class': 'h4'}):
                            date = h4.get_text()
                            date = date.split(', ')[1]
                            date = date[0:3] + date[4:6] + ' ' + str(year)
                            date = datetime.strptime(date, '%b %d %Y')
                            date = date.strftime('%m/%d/%Y')


                        # Finding data for each game
                        for game_body in game.findAll('table', attrs={'class': 'game-body'}):           
                            for num_teams, matchup in enumerate(game_body.findAll('tr', attrs={'class': 'tr'})):
                                squad = matchup.find('td', attrs={'class': 'td text team'})
                                win_percentage = matchup.find('td', attrs={'class': 'td number chance'}).get_text().strip()

                                if squad:
                                    team = squad.get_text().strip()
                                    result = ''
                                    
                                    if num_teams == 0:
                                        df = df.append(pd.Series([date, team, win_percentage, '', '', '', ''], index=df.columns), ignore_index=True)
                                    else:
                                        df = df.append(pd.Series(['', team, win_percentage, '', '', '', ''], index=df.columns), ignore_index=True)
    df.drop_duplicates(inplace=True)
    #datetime.strptime('Oct', '%b')
    #df.set_index('date')
    
    return df, year

In [116]:
def loading_odds(df, year):

    names = ['Open', 'odds new','Westgate','MGM Mirage', 'betMGM', 
             'William Hill', 'CG Technology', 'Circa Sports','Stations']
    book = pd.DataFrame(columns=names)
    output_file = 'z:\python_projects\aaa.exe'
    temp_away_list = []
    temp_home_list = []
    teams = []
    dates = []
    URL = 'http://www.vegasinsider.com/nfl/odds/las-vegas/money/'
    headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')

    for gameboard in soup.findAll('table', attrs={'class': 'viBodyContainerTble'}):
        for games in gameboard.findAll('td', attrs={'class': 'viBodyBorderNorm'}):
            for team_names in games.findAll('td', attrs={'class': 'viCellBg1 cellTextNorm cellBorderL1'}):
                for date in team_names.findAll('span', attrs={'class': 'cellTextHot'}):
                    date = date.get_text().split()[0]
                    month = date.split('/')[0]
                    day = date.split('/')[1]
                    
                    if(int(day[0]) == 0):
                        day = day[1:]
                        
                    date = str(month) + '/' + str(day) + '/' + str(year)
                    dates.extend([date, date])
                
                for team_name in team_names.findAll('a', attrs={'class': 'tabletext'}):
                    teams.append(team_name.get_text())

            for team_names in games.findAll('td', attrs={'class': 'viCellBg2 cellTextNorm cellBorderL1'}):
                for date in team_names.findAll('span', attrs={'class': 'cellTextHot'}):
                    date = date.get_text().split()[0]
                    month = date.split('/')[0]
                    day = date.split('/')[1]
                    
                    if(int(day[0]) == 0):
                        day = day[1:]
                        
                    date = str(month) + '/' + str(day) + '/' + str(year)
                    dates.extend([date, date])

                
                for team_name in team_names.findAll('a', attrs={'class': 'tabletext'}):
                    teams.append(team_name.get_text())

            for num, spread in enumerate(games.findAll('td', attrs={'class': ['viCellBg1 cellTextNorm cellBorderL1 center_text nowrap',
                                                                              'viCellBg1 cellTextHot cellBorderL1 center_text nowrap']})):           
                spread_text = spread.get_text().strip()

                if spread_text == '' or spread_text == 'XXXX':
                    away_spread = np.nan
                    home_spread = np.nan
                elif(spread_text[4] == '+' or spread_text[4] == '-'):
                    away_spread = spread_text[0:4]
                    home_spread = spread_text[4:]
                else:
                    away_spread = spread_text[0:5]
                    home_spread = spread_text[5:]

                temp_away_list.append(away_spread)
                temp_home_list.append(home_spread)
                if len(temp_home_list) == 9:
                    book = book.append(pd.Series(temp_away_list, index=names), ignore_index=True)
                    book = book.append(pd.Series(temp_home_list, index=names), ignore_index=True)
                    temp_away_list = []
                    temp_home_list = []

            for num, spread in enumerate(games.findAll('td', attrs={'class': ['viCellBg2 cellTextNorm cellBorderL1 center_text nowrap',
                                                                              'viCellBg2 cellTextHot cellBorderL1 center_text nowrap']})):           
                spread_text = spread.get_text().strip()

                if spread_text == '' or spread_text == 'XXXX':
                    away_spread = np.nan
                    home_spread = np.nan
                elif(spread_text[4] == '+' or spread_text[4] == '-'):
                    away_spread = spread_text[0:4]
                    home_spread = spread_text[4:]
                else:
                    away_spread = spread_text[0:5]
                    home_spread = spread_text[5:]
                temp_away_list.append(away_spread)
                temp_home_list.append(home_spread)

                if len(temp_home_list) == 9:
                    book = book.append(pd.Series(temp_away_list, index=names), ignore_index=True)
                    book = book.append(pd.Series(temp_home_list, index=names), ignore_index=True)
                    temp_away_list = []
                    temp_home_list = []

    book['team'] = teams[0:len(book.index)]
    print(book)
 
    book['date'] = dates
    book.set_index('team', inplace=True)

    book.dropna(how='all', thresh=5, inplace=True)
    #book = book.reindex(index=df.team)
    book['team'] = book.index
    book.replace(np.nan, '', inplace=True)
    odds = book[['odds new', 'team']]
    print(odds)
    print(df)
    #df = pd.merge(odds, df)
    return odds

In [117]:
a = loading_odds(df, year)
b = pd.merge(df, a)
b

    Open odds new Westgate MGM Mirage betMGM William Hill CG Technology  \
0   +155     +190     +185       +190   +190         +185          +180   
1   -190     -230     -230       -230   -230         -215          -210   
2   +106     -140      NaN        NaN   -140         -140          -140   
3   -117     +120      NaN        NaN   +120         +120          +120   
4   +112     +112      NaN        NaN    NaN         +125          +120   
5   -136     -136      NaN        NaN    NaN         -145          -140   
6   -113     -110      NaN        NaN   -110         -110           NaN   
7   +103     -110      NaN        NaN   -110         -110           NaN   
8   -209     -186      NaN        NaN   -190         -175          -185   
9   +185     +150      NaN        NaN   +160         +155          +165   
10  -136     -130      NaN        NaN   -130         -130          -130   
11  +123     +110      NaN        NaN   +110         +110          +110   
12  +126     +120      Na

,date,team,win%,odds,implied%,pick,w/l,odds new
0,10/3/2019,L.A. Rams,45%,105,0.487805,,,-210
1,,Seattle,55%,-125,0.555556,,,+112
2,10/6/2019,Arizona,39%,145,0.408163,,,+110
3,,Cincinnati,61%,-165,0.622642,,,+450
4,10/6/2019,Atlanta,37%,180,0.357143,Atlanta,,-130
5,,Houston,63%,-220,0.6875,,,+200
6,10/6/2019,Baltimore,58%,-180,0.642857,,,-600
7,,Pittsburgh,42%,160,0.384615,Pittsburgh,,+237
8,,Tennessee,68%,-160,0.615385,Tennessee,,+120
9,10/6/2019,Jacksonville,40%,155,0.392157,Jacksonville,,-110


In [118]:
def game_outcomes(df):
    
    
    #df_columns = ['team', 'win%', 'date', 'result']
    #df = pd.DataFrame(columns=df_columns)

    year = 2019
    result = ''
    URL = 'https://projects.fivethirtyeight.com/2019-nfl-predictions/games/?ex_cid=rrpromo'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
    source = requests.get(URL, headers=headers)
    soup = BeautifulSoup(source.content, 'html.parser')
    winners = []
    losers = []

    # Finding year of predictions
    for timestamp in soup.findAll('div', attrs={'class': 'container'}):
        for year in timestamp.findAll('div', attrs={'id': 'intro'}):
            year = int(year.h1.get_text()[0:5])

    # Gathering data
    for week in soup.findAll('section', attrs={'class': 'week'}):
        if week.get_text()[5] == '5':
            for date in week.findAll('div', attrs={'class': 'days'}):
                for day in date.findAll('div', attrs={'class': 'day'}):  
                    for game in day.findAll('div', attrs={'class': 'game'}):

                        # Finding date of each game  
                        for h4 in day.findAll('h4', attrs={'class': 'h4'}):
                            date = h4.get_text()
                            date = date.split(', ')[1]
                            date = date[0:3] + date[4:6] + ' ' + str(year)
                            date = datetime.strptime(date, '%b %d %Y')
                            date = date.strftime('%m/%d/%Y')


                        # Finding data for each game
                        for game_body in game.findAll('table', attrs={'class': 'game-body'}):           
                            for num_teams, matchup in enumerate(game_body.findAll('tr', attrs={'class': 'tr'})):
                                winner = matchup.find('td', attrs={'class': 'td text team winner'})
                                loser = matchup.find('td', attrs={'class': 'td text team loser'})

                                if winner:
                                    winners.append(winner.get_text().strip())
                                elif loser:
                                    losers.append(loser.get_text().strip())
                                else:
                                    continue
                                
    
    return winners, losers

In [119]:
winners, losers = game_outcomes(df)

df['result'] = df.apply(lambda row: team_won_lost(row, winners, losers), axis=1)
df['odds'] = df.apply(lambda row: odds_formatter(row), axis=1)
df['w/l'] = df.apply(lambda row: money_won_lost(row), axis=1)
df

ValueError: ("invalid literal for int() with base 10: ''", u'occurred at index 30')

In [ ]:
spreadsheet = pd.merge(odds, df)
spreadsheet = spreadsheet[['date', 'team', 'win%', 'result', 'odds']]
spreadsheet.set_index('date')
spreadsheet['implied%'] = spreadsheet.apply(lambda row: implied_probability(row),
                                            axis=1)
spreadsheet['pick'] = spreadsheet.apply(lambda row: pick(row), axis=1)
spreadsheet['w/l'] = spreadsheet.apply(lambda row: money_won_lost(row), axis=1)
spreadsheet = spreadsheet[['date', 'team', 'win%', 'odds', 'implied%', 
                           'pick', 'result', 'w/l']]
spreadsheet

In [ ]:
def getting_spreadsheet(file_name):
    """Loads spreadsheet
    Args:
        file_name: path of file
    Returns:
        dataframe of data
    """
    
    return(pd.read_csv(file_name))

In [ ]:
def odds_checker(row):
    
    if row['odds'] != '':
        return(row['odds'])
    else:
        return(row['odds new'])

In [ ]:
def implied_probability(row):
    """Uses odds to determine implied probability
    Args:
        row: row of data from dataframe
    Returns:
        implied win probability if it exists
    """
    if row['odds'] == '':
        return('')
    elif row['odds'][0] == '+':
        return(100.0/(100+int(row['odds'][1:])))
    elif row['odds'][0] == '-':
        return(int(row['odds'][1:])/(100.0+int(row['odds'][1:])))
    else:
        return('')

In [ ]:
def pick(row):
    """Uses win% and odds to determine what team to pick
    Args:
        row: row of data from dataframe
    Returns:
        pick if there is one
    """
    if row['implied%'] == '':
        return('')
    elif (float(row['win%'][:-1])/100.0 > row['implied%']):
        return(row.team)
    else:
        return('')
    

In [120]:
def odds_formatter(row):
    

    if int(row['odds']) >= 100:
        return('+' + str(row['odds']))
    if int(row['odds']) <= -100:
        return(str(row['odds']))
    else:
        return(str(row['odds'])) 

In [121]:
def team_won_lost(row, winners, losers):
    """Determines team outcomes
    Args:
        row: row of data from dataframe
    Returns:
        updated entry for winner/loser
    """
    if row['team'] in winners:
        return('w')
    elif row['team'] in losers:
        return('l')
    else:
        return('')

In [122]:
def money_won_lost(row):
    """Determines amount won lost
    Args:
        row: row of data from dataframe
    Returns:
        amount won/lost
    """
    if row['pick'] == '':
        return('')
    else:
        if row['result'] == 'w':
            if row['odds'][0] == '+':
                return(row['odds'][1:])
            else:
                return(100)
        elif row['result'] == 'l':
            if row['odds'][0] == '+':
                return(-100)
            else:
                return(row['odds']) 
        else:
            return('')
        

In [123]:
names = ['Open', 'odds new','Westgate','MGM Mirage', 'betMGM', 
             'William Hill', 'CG Technology', 'Circa Sports','Stations']
book = pd.DataFrame(columns=names)
output_file = 'z:\python_projects\aaa.exe'
temp_away_list = []
temp_home_list = []
teams_list = []
dates_list = []
URL = 'http://www.vegasinsider.com/nfl/odds/las-vegas/money/'
headers = {"User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}
source = requests.get(URL, headers=headers)
soup = BeautifulSoup(source.content, 'html.parser')

for gameboard in soup.findAll('table', attrs={'class': 'viBodyContainerTble'}):
    for num, games in enumerate(gameboard.findAll('td', attrs={'class': 'viBodyBorderNorm'})):
        for game_info in games.findAll('td'):#, attrs={'class': ['viCellBg1 cellTextNorm cellBorderL1',
                                             #                 'viCellBg2 cellTextNorm cellBorderL1']}):
            #print(game_info.get_text())
            for date in game_info.findAll('span', attrs={'class': 'cellTextHot'}):
                date = date.get_text().split()[0]
                month = date.split('/')[0]
                day = date.split('/')[1]

                if(int(day[0]) == 0):
                    day = day[1:]

                date = str(month) + '/' + str(day) + '/' + str(year)
                dates_list.extend([date, date])

            for team_name in game_info.findAll('a', attrs={'class': 'tabletext'}):
                teams_list.append(team_name.get_text())
                
            
            #print game_info.attrs

        for num, spread in enumerate(games.findAll('td', attrs={'class': ['viCellBg1 cellTextNorm cellBorderL1 center_text nowrap',
                                                                          'viCellBg1 cellTextHot cellBorderL1 center_text nowrap',
                                                                          'viCellBg2 cellTextNorm cellBorderL1 center_text nowrap',
                                                                          'viCellBg2 cellTextHot cellBorderL1 center_text nowrap']})):           
            spread_text = spread.get_text().strip()
            if spread_text == '' or spread_text == 'XXXX':
                away_spread = np.nan
                home_spread = np.nan
            elif(spread_text[4] == '+' or spread_text[4] == '-'):
                away_spread = spread_text[0:4]
                home_spread = spread_text[4:]
            else:
                away_spread = spread_text[0:5]
                home_spread = spread_text[5:]

            temp_away_list.append(away_spread)
            temp_home_list.append(home_spread)
            if len(temp_home_list) == 9:
                book = book.append(pd.Series(temp_away_list, index=names), ignore_index=True)
                book = book.append(pd.Series(temp_home_list, index=names), ignore_index=True)
                temp_away_list = []
                temp_home_list = []


book['team'] = teams
book['date'] = dates
odds = book[['date', 'team', 'odds new']]
odds

,date,team,odds new
0,10/7/2019,Cleveland,+190
1,10/7/2019,San Francisco,-230
2,10/10/2019,N.Y. Giants,+700
3,10/10/2019,New England,-1100
4,10/13/2019,Carolina,-140
5,10/13/2019,Tampa Bay,+120
6,10/13/2019,Cincinnati,+450
7,10/13/2019,Baltimore,-600
8,10/13/2019,Seattle,+112
9,10/13/2019,Cleveland,-136
